# Graph analysis

In [47]:
import sys
import networkx as nx
sys.path.append("../../../")

from mowl.datasets.base import PathDataset
from mowl.graph.util import parser_factory
import yaml
import pandas as pd
import logging
logging.basicConfig(level=logging.DEBUG)   
import pickle as pkl
from mowl.graph.edge import Edge

In [75]:
def main():
    parser = parser_factory(graph_method, ds.ontology, bidirectional_taxonomy=True)
    edges_path = f"data/edges_ppi_{graph_method}.pkl"
    
    try:
        logging.debug("try")
        infile = open(edges_path, 'rb')
        edges = pkl.load(infile)
    except:
        logging.debug("except")
        logging.debug("Parsing ontology...")
        edges = parser.parse()
        logging.info("Finished parsing ontology")
        edges = [(str(e.src()), str(e.rel()), str(e.dst())) for e in edges]
        outfile = open(edges_path, 'wb')
        pkl.dump(edges, outfile)

    terms_file = file_params["terms_file"]

    with open(terms_file) as f:
        removed_classes = set()
        terms = list(set(f.read().splitlines()) - removed_classes)
        edges = [Edge(s, r, d) for s,r,d in edges if s in terms and  d in terms]# and ics[s] > 0.3 and ics[d] > 0.3]
        

    
    graph = nx.DiGraph()
    
    for edge in edges:
        src, rel, dst = edge.src(), edge.rel(), edge.dst()

        graph.add_edge(src, dst)
        graph.edges[src,dst]["type"] = rel
        graph.nodes[src]["val"] = False
        graph.nodes[dst]["val"] = False
    return graph

In [76]:
def parseYAML(yaml_path):
    with open(yaml_path, 'r') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)

    return params

In [83]:
config = "config/ppi/yeast/dl2vec.yaml"

params = parseYAML(config)

graph_method = params["general"]["graph-gen-method"]
ontology = params["general"]["ontology"]
use_case = params["general"]["use-case"]

n_hidden = params["rgcn-params"]["n-hidden"]
dropout = params["rgcn-params"]["dropout"]
lr = params["rgcn-params"]["lr"]
num_bases = params["rgcn-params"]["num-bases"]
batch_size = params["rgcn-params"]["batch-size"]
epochs = params["rgcn-params"]["epochs"]
normalize =  params["rgcn-params"]["normalization"]
regularization = params["rgcn-params"]["regularization"]
self_loop =  params["rgcn-params"]["self-loop"]
seed =  params["rgcn-params"]["seed"]
min_edges = params["rgcn-params"]["min-edges"]
file_params = params["files"]



ds = PathDataset(ontology, None, None)
graph =  main()

DEBUG:root:BID IS: True
DEBUG:root:try


In [84]:
diGraph = graph.to_undirected()
[len(c) for c in sorted(nx.connected_components(diGraph), key=len, reverse=True)]

[7958]

In [85]:
[len(c) for c in sorted(nx.strongly_connected_components(graph), key=len, reverse=True)]

[6996, 962]